In [1]:
#for building the pyomo model
import pyomo.environ as pe

In [2]:
#for solving the pyomo model
import pyomo.opt as po

#### Basketball Team Problem
Cristina Hernández, Macarena Vargas, Guillermo Ruiz, Beatriz Jiménez. 3ºIMAT B 

In [3]:
#create the model
model = pe.ConcreteModel()

#### SETS

p: players [1,2,3,4,5,6,7,8,9]


z: position [guard, forward, center]

In [4]:
model.players = pe.Set(initialize=[1,2,3,4,5,6,7,8,9])
model.positions = pe.Set(initialize=['Guard', 'Forward', 'Center'])

#### Parameters
$H_{p}$: Handle of player p


$S_{p}$: Shot of player p


$R_{p}$: Rebound of player p 


$D_{p}$: defense of player p


$A_{pz}$: player p can play in position z (binary)


$NP_{z}$: minimum numer of players in position z


$AV$:minimum average value of shot, rebound and handle

In [5]:
# inicialize all the parameters: 
# parameters H, S, R , D
handle_dict = {
    1:2, 2:3, 3:2, 4:1, 5:1, 6:3, 7:3, 8:2, 9:3
}
shot_dict = {
    1:1, 2:3, 3:3, 4:3, 5:3, 6:1, 7:2, 8:1, 9:3
}
rebound_dict = {
    1:3, 2:1, 3:2, 4:3, 5:1, 6:2, 7:2, 8:3, 9:1
}
defense_dict = {
    1:3, 2:2, 3:2, 4:1, 5:2, 6:3, 7:1, 8:2, 9:3
}
model.H = pe.Param(model.players, initialize=handle_dict)
model.S = pe.Param(model.players, initialize=shot_dict)
model.R = pe.Param(model.players, initialize=rebound_dict)
model.D = pe.Param(model.players, initialize=defense_dict)
# parameter A

elig_dict = {
    # Player 1
    (1,'Guard'):0,   (1,'Forward'):0, (1,'Center'):1,
    # Player 2
    (2,'Guard'):1,   (2,'Forward'):0, (2,'Center'):0,
    # Player 3
    (3,'Guard'):0,   (3,'Forward'):1, (3,'Center'):1,
    # Player 4
    (4,'Guard'):1,   (4,'Forward'):1, (4,'Center'):0,
    # Player 5
    (5,'Guard'):0,   (5,'Forward'):1, (5,'Center'):1,
    # Player 6
    (6,'Guard'):1,   (6,'Forward'):1, (6,'Center'):0,
    # Player 7
    (7,'Guard'):0,   (7,'Forward'):1, (7,'Center'):1,
    # Player 8
    (8,'Guard'):0,   (8,'Forward'):0, (8,'Center'):1,
    # Player 9
    (9,'Guard'):0,   (9,'Forward'):1, (9,'Center'):0,
}

model.A = pe.Param(model.players, model.positions, initialize=elig_dict)

#parameter NP
NP_dict = {
    'Guard':2,
    'Forward':2,
    'Center':1
}
model.NP = pe.Param(model.positions, initialize=NP_dict)

# parameter AV
model.AV = pe.Param(initialize=2)


#### Variables

$X_{pz}$: if the player p is selected in the position z (binary)

In [6]:
model.X = pe.Var(model.players,model.positions, within = pe.Binary)


#### Objective Function

max $\sum_{pz}(D_{p} *X_{pz} )$

In [7]:
def funcion_objetivo(m):
    return sum(m.D[p] * m.X[p, z] for p in m.players for z in m.positions)

model.cost = pe.Objective(rule=funcion_objetivo, sense=pe.maximize)

#### Constrains

Limit number of players per position z

In [8]:
model.players_per_position = pe.ConstraintList()
for z in model.positions:
    model.players_per_position.add(
        sum(model.X[p, z] for p in model.players) == model.NP[z]
    )

Only available positions

In [9]:
model.available_position = pe.ConstraintList()
for p in model.players:
    for z in model.positions:
        model.available_position.add(model.X[p, z] <= model.A[p, z])

one position per player

In [10]:
model.one_pos_per_player = pe.ConstraintList()
for p in model.players:
    model.one_pos_per_player.add(
        sum(model.X[p, z] for z in model.positions) <= 1
    )

Minimum average value of handle, shot, rebound

In [11]:
team_size = sum(model.NP[z] for z in model.positions)

model.min_avg_handle = pe.ConstraintList()
model.min_avg_handle.add(
    sum(model.H[p] * model.X[p, z] for p in model.players for z in model.positions) >= team_size * model.AV
)

model.min_avg_shot = pe.ConstraintList()
model.min_avg_shot.add(
    sum(model.S[p] * model.X[p, z] for p in model.players for z in model.positions) >= team_size * model.AV
)

model.min_avg_rebound = pe.ConstraintList()
model.min_avg_rebound.add(
    sum(model.R[p] * model.X[p, z] for p in model.players for z in model.positions) >= team_size * model.AV
)

If player 3 is selected then player 6 cannot be selected

In [12]:
model.no_3_with_6 = pe.ConstraintList()
model.no_3_with_6.add(
    sum(model.X[3, z] for z in model.positions) + sum(model.X[6, z] for z in model.positions) <= 1
)

If player 1 is selected, then player 4 or 5 must be
selected but not both.

In [13]:
model.if1then4xor5 = pe.ConstraintList()
sum45 = sum(model.X[4, z] + model.X[5, z] for z in model.positions)
sum1  = sum(model.X[1, z] for z in model.positions)
model.if1then4xor5.add(sum45 <= 2 - sum1)
model.if1then4xor5.add(sum45 >=     sum1)

Player 8 or play 9 must be selected, but not both

In [14]:
model.only8or9 = pe.ConstraintList()
model.only8or9.add(
    sum(model.X[8, z] for z in model.positions) + sum(model.X[9, z] for z in model.positions) == 1
)

Solver


In [15]:
solver = pe.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

# Mostrar estado de la optimización
print("Solver Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)



Set parameter Username
Set parameter LicenseID to value 2707272
Academic license - for non-commercial use only - expires 2026-09-11
Read LP format model from file C:\Users\crist\AppData\Local\Temp\tmpfwn1hzbo.pyomo.lp
Reading time = 0.01 seconds
x1: 46 rows, 27 columns, 192 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 46 rows, 27 columns and 192 nonzeros
Model fingerprint: 0xc98609d6
Variable types: 0 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 8.0000000
Presolve removed 43 rows and 22 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 11 nonzeros
Found heuristic soluti

In [16]:
print("Optimal cost:", pe.value(model.cost))

Optimal cost: 12.0


In [17]:
for player in model.players:
    for position in model.positions:
        play = pe.value(model.X[player,position])
        if play > 1e-6:  # mostramos solo valores positivos
            print(f"The player {player} plays in the position {position}")

The player 1 plays in the position Center
The player 2 plays in the position Guard
The player 4 plays in the position Forward
The player 6 plays in the position Guard
The player 9 plays in the position Forward
